# Wind Power forecasting for the day-ahead energy market
### by Compagnie Nationale du Rhône (CNR)

#  Table of Contents
* [Framing the Problem](#frame_problem)
    - [Context](#context)
    - [Goal](#goal)
    - [Data description](#data_des)
        - [Complementary data](#comp_data)
        - [Metric](#metric)
* [Getting the Data](#getting_data)
    - [Converting the data](#convertion)
    - [Data split in traning and test sets](#train_test)

# Framing the Problem
<a id="frame_problem"></a>

## Context
<a id="context"></a>
CNR is the French leading producer of exclusively renewable energy (water, wind, sun) and the concessionary of the Rhone river for hydroelectricity production, river navigation and irrigation for agricultural use. This challenge focuses on wind energy production forecast. CNR currently owns around 50 Wind Farms (WF) for a total installed capacity of more than 600 MW. Every day, CNR sells on the energy market its wind energy production for the day ahead. In order to sell the right amount of energy, as well as for legal requirements towards the French Transmission System Operator (TSO) in charge of the electric network stability, CNR needs to know beforehand how much energy the wind farms will produce the day ahead.

## Goal
<a id="goal"></a>
The goal of this challenge is to predict the energy production of six wind farms (WF) owned by CNR. Each WF production will be individually predicted, using meteorological forecasts as input. Predictions will focus on the day-ahead energy production (hourly production forecasts from day D+1 00h to day D+2 00h).

## Data Description
<a id="data_desc"></a>

The competitors will have access to the six WF hourly production data from May the 1st of 2018 to January the 15th of 2019 (8 months and 15 days). This defines the training dataset, since day-ahead WF hourly production is the prediction target (predictand). Provided hourly WF power production consists in the raw recordings of the DSO (Distribution System Operator), and should therefore be considered as a reference, even if it could contain erroneous or suspect data. The choice is left to competitors to criticize or not this data, using complementary data provided apart.

For both training and test periods, predicting variables (predictors) will be given. It consists in hourly forecasted meteorological variables, provided by various Numerical Weather Prediction (NWP) models. NWP models are meteorological models run by several national weather prediction services. For confidentiality reasons, the name of the NWP models will not appear. They will be named with generic names NWP1, ... , NWPn.

Here is a description of all data provided in the input csv files:

- *ID*: This is the unique ID of each row in the csv files. One ID correspond to a couple Time / WF. The ID of the test set are consecutive to the ID of the training set.

- *WF*: The considered Wind Farm. WF ranges from WF1 to WF6. It is crucial for the competitors to be aware that this prediction problem is totally dependent to the WF considered. In other words, the statistical link between input variables and wind power production is completely different from one WF to another. Consequently, **it could be judicious to train specific prediction algorithms for each WF, instead of training a unique algorithm which could be unable to model the behavior of each WF**.

- *Time* (UTC): date and hour of the target timestep, i.e. corresponding to the observed Power production. Time zone is Coordinated Universal Time (UTC).

- *Meteorological variables*: Numerical Weather Predictions are provided by meteorological centers several times a day (updates), typically at 00h UTC, 06h UTC, 12h UTC and 18h UTC. We call these sets of forecasts "Runs". Consequently, if the input file contains forecasts arising from several runs, this implies that a single NWP is associated with several forecasts for the same forecasting time. Therefore, the information on the hour of run is provided.

The format of the header of the csv files for the meteorological variables is the following: *NWPi_HourOfTheRun_DayOfTheRun_Variable*, with:

- *NWPi*: the considered Numerical Weather Prediction model (meteorological model);

- *HourOfTheRun*: the hour (UTC) of the considered run. According to the NWP, it could be 00h, 06h, 12h and 18h (case of NWP with 4 runs per day) or only 00h and 12h (case of NWP with 2 runs per day);

- *DayOfTheRun*: the day of the considered run. We provide in the csv files predictions from the D-2 day runs (the day before yesterday), D-1 day runs (yesterday) and D day runs;

- *Variable*: the different meteorological variables forecasted by the NWP:

    - *U* and *V* components of the wind at 100m (or 10m) height (m/s). These are the zonal and meridional velocities of the wind, respectively. Both are given at a height of 100m above ground for NWP1, NWP2 and NWP3. U and V are given at a height of 10m for NWP4. Even if these variables are given at hourly timestep, **we draw competitors attention on the fact that the temporal representativity of the given values is for a 10-minutes window ranging from H-10 min to H**.

    - *T*: Temperature of air (°C). This is the averaged temperature over the entire hour (from H-1 to H). Wind power production is sensitive to air temperature since it affects the air density. This variable is provided only for NWP1 and NWP3.

    - *CLCT*: Total cloud cover (%).This is the total cloud cover of the sky, ranging from 0% (clear sky, no cloud) to 100% (fully clouded sky). The value is an instant value at hour H. This variable is provided only for NWP4.

    - *Observed Power Production (MW or MW.h)*: this is the observed total amount of energy injected by the WF to the electric network over the entire hour H-1 to H (MW.h). Equivalently, we can consider that this is the mean power output of the WF observed between H-1 and H (MW).

### Complementary data
<a id="comp_data"></a>

We provide complementary data in the `.zip` supplementary files. These data may be used by the competitors to prepare or criticize WF hourly production data, but they are not predictors. The file `WindFarms_complementary_data.csv` contains the following hourly variables:

* Average power output for each wind turbine of the WF (MW)

* Cumulated energy produced by each wind turbine (MWh). This value could differ from the hourly average power output when the considered turbine has not been operational during the entire hour.

* Observed average wind direction at hub (nacelle) height for each wind turbine (°, from 0 to 359)

* Observed average wind speed at hub (nacelle) height for each wind turbine (m/s)

* Observed average nacelle direction for each wind turbine (°, from 0 to 359)

* Observed average rotational speed of each wind turbine (s$^{-1}$)

### Metric
<a id="metric"></a>

The metric used to rank the predicting performance is a relative form of the absolute error. We call it the CAPE (Cumulated Absolute Percentage Error). The formulation of CAPE for one WF would be the following:

$\text{CAPE}_{k}\left( {\widehat{Y}}_{k},Y_{k} \right) = 100 \times \frac{\sum_{i = 1}^{N_{k}}\left| Y_{i,k} - {\widehat{Y}}_{i,k} \right|}{\sum_{i = 1}^{N_{k}}Y_{i,k}}$

With $\text{CAPE}_{k}$ the metric for the WF $k$ (%),
$N_{k}$ the length of the test sample for WF $k$ only,
$Y_{i,k}$ the observed production for WF $k$ and hour $i$ (MW or MW.h),
and $\widehat{Y}_{i,k}$ the predicted production for WF $k$ and hour $i$ (MW or MW.h).

For convenience reasons, data relative to the 6 WF have been regrouped in the same train and test input files. Therefore, the metric used in the challenge is the overall average CAPE for the 6 WF, calculated as:

$\text{CAPE}\left( \widehat{Y},Y \right) = 100 \times \frac{\sum_{i = 1}^{M}\left| Y_{i} - {\widehat{Y}}_{i} \right|}{\sum_{i = 1}^{M}Y_{i}}$

With $M$ the length of the test sample for all the 6 WF ($M$ is the sum of $N_{k}$ for all $k$).

This formulation results in a non-homogeneous contribution of all the WF to the final value of CAPE: CAPE will be more sensitive to WF with the highest energy production values.

In [ ]:
# Libraries
%load_ext autoreload
%autoreload 2

# Common libraries
import numpy as np
import pandas as pd
import os
import datetime as dt

# Graphics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import plotly as pty
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

import re
from collections import OrderedDict

# Save images 
DIR = "../../TFM/reports/figures/"
WF = "WF1"
IMAGES_PATH = os.path.join(DIR, WF)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore warnings (SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


In [ ]:
# Import data
X_train = context.catalog.load("X_train_raw")
Y_train = context.catalog.load("y_train_raw")
X_test = context.catalog.load("X_test_raw")
# extra_data = pd.read_csv('../data/external/WindFarms_complementary_data.csv', sep=';', parse_dates=['Time (UTC)'])

# Parse dates
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M')
X_train['Time'] = pd.to_datetime(X_train['Time'], format='%d/%m/%Y %H:%M')
X_test['Time'] = pd.to_datetime(X_test['Time'], format='%d/%m/%Y %H:%M')

In [ ]:
%store X_train
%store Y_train

### Data split into training and test sets
<a id="train_test" />

In order to test the models localy, before making submissions to the challenge, we're splitting `train` dataframe created above, into traning and test dataframes. We'll get for testing the last two days of `Time` column for all the WF's; the rest of the `training` dataframe will be for traning.

In [ ]:
def split_data_by_date(date, X, y):
    """
    It splits X and y sets by a 'Time' value 
    into sets for training and testing. 
        - Return: a dictionary with the four sets
                  (X_train, y_train, X_test, y_test)
    """
    sets = {}
    date_cut = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    
    X_test = X[X['Time'] > date_cut]
    X_train = X[X['Time'] <= date_cut]
    y_train = y[y.ID.isin(X_train.ID)]
    y_test = y[y.ID.isin(X_test.ID)]
    
    sets['X_train'] = X_train
    sets['X_test'] = X_test
    sets['y_train'] = y_train
    sets['y_test'] = y_test
    
    return sets

In [ ]:
train_test_dfs = split_data_by_date('2018-11-13 23:00:00', X_train, Y_train)
X_train_2 = train_test_dfs.get('X_train')
X_test_2 = train_test_dfs.get('X_test')
Y_train_2 = train_test_dfs.get('y_train')
Y_test_2 = train_test_dfs.get('y_test')

In [ ]:
X_train_2.tail()

In [ ]:
X_test_2.head()

In [ ]:
X_test_2.tail()

In [ ]:
Y_train_2

In [ ]:
Y_train_2['Production'].describe()

In [ ]:
Y_test_2.head()

In [ ]:
%store X_train_2
%store Y_train_2
%store X_test_2
%store Y_test_2